## João Marcos Machado Couto
## Matricula: 2017014421
### Link do vídeo: https://youtu.be/OsRTctj1IFI

In [ ]:
import cv2
import numpy as np

from OpenGL.GLU import *
from OpenGL import *
from OpenGL.GLUT import *

from PIL import Image

from objloader import *
import matplotlib.pyplot as plt

import time


pikachuOn = True
cubesOn = True

# Calibração da Camera

Feita via "Camera Calibration Toolbox for Matlab" de Jean-Yves Bouguet (http://www.vision.caltech.edu/bouguetj/calib_doc/)

Capturei frames 4 aleátorios do vídeo disponibilizado para fazer a calibração

A seguir, os parametros obtidos e suas respectivas incertezas:

Focal Length:          fc = [ 823.02785   857.55218 ] +/- [ 27.59011   25.29775 ]

Principal point:       cc = [ 566.92369   396.42114 ] +/- [ 12.48074   22.10106 ]

Skew:             alpha_c = [ 0.00000 ] +/- [ 0.00000  ]   => angle of pixel axes = 90.00000 +/- 0.00000 degrees

Distortion:            kc = [ 0.09076   -0.21946   -0.00693   -0.00024  0.00000 ] +/- [ 0.07284   0.50725   0.00776   0.00641  0.00000 ]

Pixel error:          err = [ 0.37915   0.42384 ]

Com essa saída definimos a matriz de parametros intrinsecos da câmera:

In [ ]:
intrinsicMatrix = np.array([
                            [823.02785, 0.0, 320], 
                            [0.0, 857.55218, 240], 
                            [0.0, 0.0, 1.0]
                            ])
cameraDistortion = [ 0.09076, -0.21946 ,-0.00693 ,-0.00024, 0.00000 ] 



# Leitura de inputs (alvo+video)

###  O Video

#####  leitura do vídeo de input via OpenCV

Ref: https://theailearner.com/2018/10/15/extracting-and-saving-video-frames-using-opencv-python/

Ref2: https://stackoverflow.com/questions/33311153/python-extracting-and-saving-video-frames

In [ ]:
inputVideo = cv2.VideoCapture('entrada.mp4')

##### Captura e decodificação de frames

In [ ]:
#Read retorna False, NONE quando não consegue ler mais conteúdo
frames = []
success,image = inputVideo.read()
while success:
    frames.append(image)
    success, image = inputVideo.read()

### O alvo

##### Leitura e binarização do alvo

In [ ]:
alvo0rot = cv2.imread('alvo.jpg', 0)
_, alvo0rot = cv2.threshold(alvo0rot, 127, 255, cv2.THRESH_BINARY)

##### Rotações do alvo

Ref: https://www.geeksforgeeks.org/python-opencv-cv2-rotate-method/

In [ ]:
alvo1rot = cv2.rotate(alvo0rot, cv2.ROTATE_90_CLOCKWISE) 
alvo2rot = cv2.rotate(alvo1rot, cv2.ROTATE_90_CLOCKWISE) 
alvo3rot = cv2.rotate(alvo2rot, cv2.ROTATE_90_CLOCKWISE) 
alvos = [alvo0rot, alvo1rot,alvo2rot,alvo3rot]

In [ ]:
#Lista auxiliar que permite acesso rápido às coordenadas dos 4 pontos de cada rotação do alvo
pontosAlvo =[]
pontosAlvo.append(np.float32([[-1, 1, 0], [-1, -1, 0], [1, -1, 0], [1, 1, 0]]))
pontosAlvo.append(np.float32([[-1, -1, 0], [1, -1, 0], [1, 1, 0], [-1, 1, 0]]))
pontosAlvo.append(np.float32([[1, -1, 0], [1, 1, 0], [-1, 1, 0], [-1, -1, 0]]))
pontosAlvo.append(np.float32([[1, 1, 0], [-1, 1, 0], [-1, -1, 0], [1, -1, 0]]))

# Determinando a posição e orientação do alvo

### Extração de bordas e contornos

Extração de bordas (B&W -> Binarização -> Bordas)

Ref: https://docs.opencv.org/master/da/d22/tutorial_py_canny.html

In [ ]:
#Um grayscale é criado antes de fazer a binarização
def binarize(image):
    grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binarizado = cv2.threshold(grayscale, 127, 255, cv2.THRESH_BINARY)
    return binarizado
    

In [ ]:
#Dado um frame binarizado, extrai as bordas da imagem
def extract_edges_given_binarized(binarizado):
    imageEdges = cv2.Canny(binarizado, 100, 200)
    return imageEdges
    

In [ ]:
print(cv2.TM_SQDIFF)

Extração de contornos

Ref: https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_contours/py_contours_begin/py_contours_begin.html#contours-getting-started

In [ ]:
#Encontra os contornos de um frame dado o frame passado pelo detector de bordas canny
def extract_contours_given_edges(imageEdges):
    contorno, _  = cv2.findContours(imageEdges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contorno

### Identificação de quadrilateros na imagem

Ref: https://stackoverflow.com/questions/55169645/square-detection-in-image/

Ref2: https://stackoverflow.com/questions/61166180/detect-rectangles-in-opencv-4-2-0-using-python-3-7

In [ ]:
#Encontra os quadrilateros de um frame dado seus contornos
#PS: "square" foi uma abuso da palavra pois na verdade neste ponto identifico quadrilateros
def find_squares_given_contours(edgesContours):
    squares = list()
    for contour in edgesContours:
        epsilon = 0.05*cv2.arcLength(contour,True)
        polygon = cv2.approxPolyDP(contour, epsilon, True)
        if len(polygon) == 4 and cv2.isContourConvex(polygon):
            squares.append(polygon)
    return squares

In [ ]:
#Só uma função auxiliar que retorna as coordenadas das quinas do alvo
def template_corners_coords(template):
    s = template.shape
    return np.float32([[0,0], [0, s[0]], [s[1], s[0]], [s[1], 0]])

### Homografia, perspective shift e identificação de quadilateros alvos

Faz a homografia e identificação dos quadrilateros que efetivamente dão match com alguma rotação do alvo

#### Utilização do método CV_TM_SQDIFF: 

#### $R(x,y) = \sum(T(x',y') - I(x + x', y + y'))^2$  para calculo da diferença entre cada quadrilatero e as rotações do alvo

Retorna uma tupla de listas: ([contorno dos quadrados que deram match], [orientação deles])


Experimentei boa parte dos métodos oferecidos pelo openCV para o calculo da diferença entre uma imagem e um template, a raíz da diferença quadrática provou-se eficaz na separação entre quadrilateros matching e não matching

In [ ]:
def homography(corner_coords,quadrados, image):
    quadrados_matched = [] #Armazena quadrilateros que deram match com alguma rotação do alvo
    orientacao = [] #Armazena a orientação do alvo com o qual um dado quadrilatero deu match
    for q in quadrados: 
        image_homography, _ = cv2.findHomography(np.float32(q), corner_coords, cv2.RANSAC)
        warped = cv2.warpPerspective(image, image_homography, alvo0rot.shape) 
        leng = warped.size
        
        diffs = [0] * len(alvos) #Diff armazena a raiz da diferença quadratica entre um quadrilatero e cada rotação do alvo
        for i,rot in enumerate(alvos):
            delta = cv2.matchTemplate(warped,rot,0)
            diffs[i] = np.sqrt(delta)

        if min(diffs) < 30000 : #Se alguma das entradas no diff for menor que este threshold considera-se um match
            quadrados_matched.append(q)
            orientacao.append(diffs.index(min(diffs)))
            

    return (quadrados_matched, orientacao)
    


# Efetivando identificação de alvos em todos os frames do vídeo

Itera sobre todos os frames capturados do video e então os processa com as funções na ordem que vimos acima

Binary -> Edges -> Contours -> "Squares" -> Homografia -> Matching

In [ ]:
cornerCoords = template_corners_coords(alvo0rot)
cena = []
for frame in frames:
    binarizado = binarize(frame)
    edges = extract_edges_given_binarized(binarizado)
    contorno = extract_contours_given_edges(edges)
    squares = find_squares_given_contours(contorno)
    homo = homography(cornerCoords,squares, binarizado)
    cena.append(homo)

    

# Determinando os parâmetros extrínsecos

Ref: https://docs.opencv.org/master/d7/d53/tutorial_py_pose.html

Faz a estimativa dos parametros extrinsecos

Código quase identico ao explicitado em sala

In [ ]:
def pose(polygon, direction):   

    _, rot, trans = cv2.solvePnP(pontosAlvo[direction], np.float32(polygon), intrinsicMatrix, np.float32(cameraDistortion))
    rodRot, _ = cv2.Rodrigues(rot)
    
    
    matriz_pose = np.append(rodRot, trans, axis=1)
    lastRow = [[0,0,0,1]]
    matriz_pose = np.append(matriz_pose, lastRow, axis = 0)
    
       
    matriz_pose[1, 0] =  matriz_pose[1, 0] * -1
    matriz_pose[2, 0] = matriz_pose[2, 0] * -1 
    matriz_pose[1, 1] = matriz_pose[1, 1] * -1
    matriz_pose[2, 1] = matriz_pose[2, 1] * -1
    matriz_pose[1, 2] = matriz_pose[1, 2] * -1
    matriz_pose[2, 2] = matriz_pose[2, 2] * -1
    matriz_pose[1, 3] = matriz_pose[1, 3] * -1
    matriz_pose[2, 3] = matriz_pose[2, 3] * -1
    
    return np.transpose(matriz_pose)

# Renderização via OpenGL

Ref: https://www.youtube.com/watch?v=M4qFGp5muVg&feature=youtu.be

Função auxiliar para a obtenção de arestas de um cubo dado as cordenadas de seus vertices

In [ ]:

def differ_counter(v1,v2):
    count = 0
    for i in range(len(v1)):
        if v1[i] != v2[i]:
            count = count + 1
    return count
        

### Renderização: cubo

In [ ]:
def cube(): 
    vertices=(
        (1, -1, -1), (1, 1, -1), (-1, 1, -1),(-1, -1, -1),
        (1, -1, 1), (1, 1, 1),(-1, -1, 1), (-1, 1, 1)
    )
    
    arestas = []
    #Em nosso cubo, vertices tem arestas entre si apenas se se diferem por apenas uma coordenada
    #Essa sequencia de comandos constroi tupla de aresta apartir desta propriedade
    for i,v1 in enumerate(vertices):
        for j,v2 in enumerate(vertices):
            c = differ_counter(v1,v2)
            if (c ==1):
                arestas.append(sorted(list([i,j])))
    arestas = [tuple(x) for x in set(tuple(x) for x in arestas)]
    arestas.remove((1,2)) 
    
           
    #Desenha a aresta indicativa da direção do cubo
    glPushAttrib(GL_CURRENT_BIT)
    glLineWidth(4)
    glBegin(GL_LINES)
    glColor3f(0., 0., 255/255)
    glVertex3fv(vertices[1])
    glVertex3fv(vertices[2])
    
    #Desenha o resto das arestas
    glColor3f(255/255, 255/255, 255/255)
    for aresta in arestas:
        for vertice in aresta:
            glVertex3fv(vertices[vertice])

    glEnd()

    glPopAttrib()

### Renderização central: chama tanto pikachu quanto os cubos

In [ ]:
def render():
    for p, d in zip([c[0] for c in cena][current_frame], [c[1] for c in cena][current_frame]):
        glLoadMatrixf(pose(p, d)) 
        #Optei por renderizar os dois (pikachu e cubos) ao mesmo tempo pra facilitar a demonstração
        
        #Se for do interesse ver um deles de cada vez
        ## basta setar cuberOn = Falsa ou pikachuOn=False na primeira celula
        
        if cubesOn: 
            cube()     
        if pikachuOn:
            glCallList(pikachu.gl_list) 

### Inicialização "standard" do openGL

In [ ]:
def init_open_gl():
    glClearColor(0, 0, 0, 0) # Setando preto como cor de limpeza da tela
    glClearDepth(1.0) 
    glEnable(GL_DEPTH_TEST) 
    glMatrixMode(GL_PROJECTION) 
    glLoadIdentity() 
    
    fovy = 2 * np.arctan(0.5 * 480 / intrinsicMatrix[1, 1]) * 180 / np.pi
    aspect = 640 * intrinsicMatrix[1, 1] / (480 * intrinsicMatrix[0, 0])
    gluPerspective(fovy, aspect, 0.1, 100.0)
    


### Renderização: background

Ref: https://www.youtube.com/watch?v=n4k7ANAFsIQ
Ref2: https://www.youtube.com/watch?v=HOZA2ph4UuE&feature=youtu.be

In [ ]:
def background(img):

    (width, height) = (640,480)
     
    
    # Ativação da textura
    background_id = glGenTextures(1)
    glBindTexture(GL_TEXTURE_2D, background_id)
    
    #Conversão a RGB utilizada pelo openGL e flip como visto em aula
    background = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    background = cv2.flip(background, 0)
    

    # Criando a textura junto ao openGL
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, width, height, 0, GL_RGB, GL_UNSIGNED_BYTE, background)

    # Desabilitando o glDepthMask para evitar que o fundo fique no topo
    glDepthMask(GL_FALSE)
    
    glMatrixMode(GL_PROJECTION)
    glPushMatrix()
    glLoadIdentity()
    gluOrtho2D(0, width, 0, height)
    
    #Ligando a textura ao background
    glEnable(GL_TEXTURE_2D)
    glBindTexture(GL_TEXTURE_2D, background_id)
    glMatrixMode(GL_MODELVIEW)
    glPushMatrix()

    #Desenha quadrilatero que ocupa a janela na integra
    glBegin(GL_QUADS)
    glTexCoord2f(0, 0); glVertex2f(0, 0)
    glTexCoord2f(1, 0); glVertex2f(width, 0)
    glTexCoord2f(1, 1); glVertex2f(width, height)
    glTexCoord2f(0, 1); glVertex2f(0, height)
    glEnd()
    
    glPopMatrix()
    glMatrixMode(GL_PROJECTION)
    glPopMatrix()
    glMatrixMode(GL_MODELVIEW)
    
    # Desligamento da textura e flush
    glBindTexture(GL_TEXTURE_2D, 0)
    glDepthMask(GL_TRUE) #Impede que o fundo fique acima do objetos
    glFlush()
 #Carrega o background. Recebe o frame da imagem (img)

    

### Renderização: callback

In [ ]:
current_frame = 0

def display_callback(pikachu):
    global current_frame #Utilizada para conseguir acessar globalmente qual é o index do proximo frame
    glMatrixMode(GL_MODELVIEW) 
    glLoadIdentity() 
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT) # Limpando buffers de cor/depth

    background(frames[current_frame]) #Renderiza o background no frame atual
    current_frame = (current_frame + 1) % len([c[0] for c in cena]) #Atualiza o index to frame atual para o proximo

    glMatrixMode(GL_PROJECTION)
    glLoadIdentity() 

    fovy = 2 * np.arctan(0.5 * 480 / intrinsicMatrix[1, 1]) * 180 / np.pi
    aspect = 640 * intrinsicMatrix[1, 1] / (480 * intrinsicMatrix[0, 0])
    gluPerspective(fovy, aspect, 0.1, 100.0)

    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()
    glEnable(GL_TEXTURE_2D)

    render()
    
    glutSwapBuffers() #Renderiza na tela o que está atualmente em buffer

### Inicialização e loop main

In [ ]:
glutInit() #
glutInitDisplayMode(GLUT_RGBA | GLUT_DOUBLE) #Utilizaremos double buffer e espaço de cor RGB
glutSetOption(GLUT_ACTION_ON_WINDOW_CLOSE, GLUT_ACTION_CONTINUE_EXECUTION) 
glutInitWindowSize(640, 480) 
glutCreateWindow(b'ICV TP2 - AR') 

init_open_gl()

In [ ]:
pikachu = OBJ('Pikachu.obj', swapyz=True)
display = lambda : display_callback(pikachu)
glutDisplayFunc(display)
glutIdleFunc(glutPostRedisplay) #Função a ser executada quando "nada esta acontecendo"

glutMainLoop()